In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. answer questions using only the following context. if you don't know the answer just say you don't know, don't make it up: \n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

In [2]:
chain.invoke("Is Aaronson guilty?")

AIMessage(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [3]:
chain.invoke("What message did he write in the table?")

AIMessage(content='He wrote "2+2=5" in the dust on the table.')

In [4]:
chain.invoke("Who is Julia?")

AIMessage(content='Julia is a character mentioned in the text. She is someone who the main character, Winston, has a deep connection with and cares about.')